# Uso de RDD's en Spark

## Creación de un RDD 
Lo primero es obtener el objeto SparkContext. Recomendación personal: Hacerlo a partir del objeto SparkSession

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

### A partir de una colección
Para crear un RDD a partir de una colección se usa el método parallelize. A continuación se muestran dos ejemplos: Uno para enteros y otro para strings

In [2]:
rdd_int = sc.parallelize(range(10))
rdd_st = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split())

### A partir de fuentes de datos
Para ello tenemos dos opciones:
- textfile: Para crear un RDD a partir de un archivo de texto
- wholeTextFiles: Para crear un RDD a partir de varios archivos de texto

In [4]:
rdd_file = sc.textFile("data/flight-data/csv/2015-summary.csv")
rdd_whole_files = sc.wholeTextFiles("data/flight-data/csv")

## Acciones
A continuación vamos a ver algunas de las acciones más frecuentes:
### Collect
Permite mostrar todos los elementos de un RDD


In [3]:
print (rdd_int.collect())
print (rdd_st.collect())
#print (rdd_file.collect())
#print (rdd_whole_files.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
['Big', 'Data', 'aplicado.', 'Curso', 'de', 'especialización', 'de', 'Inteligencia', 'Artificial', 'y', 'Big', 'Data']


### Take
Permite obtener un número determinado de elmentos del RDD

In [16]:
print (rdd_int.take(3))
print (rdd_st.take(3))
print (rdd_file.take(2))
#print (rdd_whole_files.take(2))

[0, 1, 2]
['Big', 'Data', 'aplicado.']
['DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count', 'United States,Romania,15']


### Count
Devuelve el número de elementos de un RDD

In [13]:
print (rdd_int.count())
print (rdd_st.count())
print (rdd_file.count())
print (rdd_whole_files.count())

10
12
257
6


In [9]:
rdd_st.count()

12

### Reduce
Permite, mediante una función especificada por el programador reducir el RDD a un único valor.


In [25]:
# Sumamos todos los elementos
print(rdd_int.reduce (lambda x,y: x+y))

def word_length_reducer(word1,word2):
    if (len(word1) > len (word2)):
        return word1
    else:
        return word2


print (rdd_st.reduce (word_length_reducer))
print ( rdd_file.reduce (word_length_reducer))

45
especialización
"Bonaire, Sint Eustatius, and Saba",United States,58


### First
Devuelve el primer elemento de un RDD

In [7]:
rdd_int.first()

0

In [8]:
rdd_st.first()

'Big'

### Max/min
Devuelve el valor máximo/mínimo de un RDD



In [9]:
rdd_int.min()

0

In [10]:
rdd_st.max()

'y'

## Transformaciones
### Map
Permite aplicar una función especificada por el programador a cada uno de los elementos del RDD devolviendo un RDD del mismo tamaño que el original

In [12]:
rdd_int.map (lambda x: 2*x).collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [15]:
rdd_st.map (lambda x: x.upper()).collect()

['BIG',
 'DATA',
 'APLICADO.',
 'CURSO',
 'DE',
 'ESPECIALIZACIÓN',
 'DE',
 'INTELIGENCIA',
 'ARTIFICIAL',
 'Y',
 'BIG',
 'DATA']

### Distinct
Elimina los duplicados

In [16]:
rdd_st.distinct().collect()

['Curso',
 'aplicado.',
 'especialización',
 'Big',
 'de',
 'y',
 'Inteligencia',
 'Artificial',
 'Data']

### Filter
Permite seleccionar los elementos del RDD que cumplen determinada condición


In [17]:
rdd_int.filter(lambda x: x % 2 == 0).collect()

[0, 2, 4, 6, 8]

In [20]:
rdd_st.filter(lambda x: len(x) >= 5).collect()

['aplicado.', 'Curso', 'especialización', 'Inteligencia', 'Artificial']

### SortBy
Permite reordenar el RDD en función de un criterio que puede ser especificado mediante una función lambda

In [23]:
rdd_st.sortBy(lambda x: len(x)).collect()

['y',
 'de',
 'de',
 'Big',
 'Big',
 'Data',
 'Data',
 'Curso',
 'aplicado.',
 'Artificial',
 'Inteligencia',
 'especialización']

### Randomsplit
Permite dividir un RDD convirtiéndolo en un array de RDD’s en función de un array de pesos especificado por el programador

In [21]:
for rdd in rdd_int.randomSplit([0.4, 0.6]):
    print(rdd.collect())

for rdd in rdd_st.randomSplit([0.5,0.5]):
    print(rdd.collect())


[0, 3, 6]
[1, 2, 4, 5, 7, 8, 9]
['Big', 'Data', 'aplicado.', 'Curso', 'de', 'especialización', 'y']
['de', 'Inteligencia', 'Artificial', 'Big', 'Data']


### FlatMap
Permite realizar operaciones map que no sean 1:1. Por ejemplo, al leer un archivo de texto, por defecto, se almacena cada lína como un elemento del RDD. Con flatMap podemos transformar el RDD para que cada elemento sea una palabra. En este caso, separamos por ','


In [4]:
rdd_file.flatMap(lambda x: x.split(',')).collect()

NameError: name 'rdd_file' is not defined